In [43]:
#Enter you own user and app info here, before running this project
username = "" #ADD YOUR TELEGRAM USERNAME
phone_number = "" #ADD YOUR TELEGRAM PHONE NUMBER
api_id = 0 #ADD YOUR TELEGRAM DEVELOPER APP ID
api_hash = "" #ADD YOUR TELEGRAM DEVELOPER APP HASH

#Select which trained model to use to predict results
#Can select from [formspring, wiki, twitter, sentiment104]
data = "wiki"
output_folder_name = "results/"

In [44]:
import logging
logging.basicConfig(format='[%(levelname) 5s/%(asctime)s] %(name)s: %(message)s',
                    level=logging.WARNING)

In [45]:
import os
os.environ['KERAS_BACKEND']='theano'
from AttentionLayer import AttLayer
from keras.models import load_model
import numpy as np
from tensorflow.contrib import learn

#Loading the learned Vocabulary Processor from the file system
vocab_filename = output_folder_name + data + "_vbp.pkl"
vocab_processor = learn.preprocessing.VocabularyProcessor.restore(vocab_filename)
    
#Loading the trained model from the file system
model_filename = output_folder_name + data + ".model"
model = load_model(model_filename, custom_objects={'AttLayer': AttLayer})

#Returns the predicted Sentiment Score by the model
def getSentimentValue(test_x):
    testX = np.array(list(vocab_processor.transform([test_x])))
    temp = model.predict(testX)
    return temp[0][0]

In [49]:
from telethon import TelegramClient, events
#Contains all the conversations and the messages inside for the user, since the application starts running
myDict = {}

async def signInTelegram(username, phone):
    client = TelegramClient(username, api_id, api_hash)
    
    await client.start(phone)
    async with client:
        # Getting information about yourself
        me = await client.get_me()
            
        print("Your session has been established with Telegram, now you can do chat messages from Telegram")
        print("1. In order to find out the sentiment score, you can send msg in this format: <Score>")
        print("2. Ïn order to close this session, just stop executing this cell")
        #This is called as soon as there is a new message in any conversatioon
        @client.on(events.NewMessage)
        async def my_event_handler(event):
            new_msg = event.raw_text.lower()
            if(new_msg != ''):
                if(not event.chat_id in myDict):
                     myDict[event.chat_id] = []
                     
                #We append the new chat message with the conversation id in this dictionary
                myDict[event.chat_id].append(new_msg)
                
                #If the new msg is "<score>", we calculate the sentiment for the all the existing msgs in that conversation
                if(new_msg == "<score>"): 
                    #Concatenating all the existing messages
                    concat_text = ''
                    for i in range(0, (len(myDict[event.chat_id]) - 1) ):
                        concat_text += myDict[event.chat_id][i] + ". "
                        
                    #We also removes all the chats messages for which the score is already calculated   
                    myDict[event.chat_id] = []
                    sentiment = getSentimentValue(concat_text)
                    
                    sentimentReply = ''
                    score = str("%.2f" % (sentiment * 100)) + '%';

                    if(sentiment <= 0.25):
                        sentimentReply = "\U0001F621 Negative Score: " + score  
                    elif(sentiment >= 0.75):
                        sentimentReply = "\U0001F600 Positive Score: " + score
                    else:
                        sentimentReply = "\U0001F633 Neutral Score: " + score

                    await event.reply(sentimentReply)
                
        await client.start()
        await client.run_until_disconnected()
        
await signInTelegram(username, phone_number)

Your session has been established with Telegram, now you can do chat messages from Telegram
1. In order to find out the sentiment score, you can send msg in this format: <Score>
2. Ïn order to close this session, just stop executing this cell


[WARNING/2019-07-31 21:15:11,069] tensorflow: From C:\Users\Ali\Anaconda3\lib\site-packages\tensorflow\contrib\learn\python\learn\preprocessing\text.py:203: tokenizer (from tensorflow.contrib.learn.python.learn.preprocessing.text) is deprecated and will be removed in a future version.
Instructions for updating:
Please use tensorflow/transform or tf.data.


CancelledError: 